In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import shutil

import torch, torchvision, torchvision.transforms.v2 as transforms
from torch.utils.data import DataLoader

from keras.utils import image_dataset_from_directory

import os
from tqdm import tqdm
import torch
from torchvision import datasets, transforms
from torchvision.utils import save_image
from PIL import Image

from torchvision import datasets, transforms

import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageEnhance


In [2]:
# Defining paths

TRAIN_DATA_DIR = os.path.join("data/train")
VAL_DATA_DIR = os.path.join("data/val")

CLEAN_TRAIN_DATA_DIR = os.path.join("clean/train")
CLEAN_VAL_DATA_DIR = os.path.join("clean/val")

OUTPUT_BASE = "data_augmented"
os.makedirs(OUTPUT_BASE, exist_ok=True)

In [3]:
# --- CONFIG ---
IMG_SIZE = (150, 150)
OUTPUT_BASE = "data_augmented"
SINGLE_DIR = os.path.join(OUTPUT_BASE, "single_various_augmentation")
MULTI_DIR = os.path.join(OUTPUT_BASE, "multiple_various_augmentation")

# make sure base directories exist
os.makedirs(SINGLE_DIR, exist_ok=True)
os.makedirs(MULTI_DIR, exist_ok=True)

In [4]:
def adjust_brightness_contrast(image, brightness=1.0, contrast=1.0):
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    enhancer_b = ImageEnhance.Brightness(pil_img)
    image = enhancer_b.enhance(brightness)
    enhancer_c = ImageEnhance.Contrast(image)
    image = enhancer_c.enhance(contrast)
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

def add_gaussian_noise(image, sigma=0.02):
    noise = np.random.randn(*image.shape) * 255 * sigma
    noisy = np.clip(image + noise, 0, 255).astype(np.uint8)
    return noisy

def ensure_color_channels(img):
    if len(img.shape) == 2:  # grayscale
        return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img

def augment_single(img):
    """Apply single augmentations individually."""
    aug_images = []

    # Rotation
    for angle in range(-30, 31, 10):
        if angle == 0: continue
        M = cv2.getRotationMatrix2D((IMG_SIZE[0]//2, IMG_SIZE[1]//2), angle, 1)
        aug_images.append(cv2.warpAffine(img, M, IMG_SIZE))

    # Translation
    h, w = IMG_SIZE
    for shift in [-0.1, -0.05, 0.05, 0.1]:
        M = np.float32([[1, 0, shift * w], [0, 1, shift * h]])
        aug_images.append(cv2.warpAffine(img, M, IMG_SIZE))

    # Scaling
    for scale in [0.8, 0.9, 1.1, 1.2]:
        aug_images.append(cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR))

    # Flips
    aug_images.append(cv2.flip(img, 1))  # horizontal
    aug_images.append(cv2.flip(img, 0))  # vertical

    # Shear
    for shear in [-20, -10, 10, 20]:
        M = np.array([[1, np.tan(np.radians(shear)), 0],
                      [0, 1, 0]], dtype=float)
        aug_images.append(cv2.warpAffine(img, M, IMG_SIZE))

    # Brightness & Contrast
    for b in [0.8, 0.9, 1.1, 1.2]:
        aug_images.append(adjust_brightness_contrast(img, brightness=b))
    for c in [0.8, 0.9, 1.1, 1.2]:
        aug_images.append(adjust_brightness_contrast(img, contrast=c))

    # Noise
    for sigma in [0.01, 0.03, 0.05]:
        aug_images.append(add_gaussian_noise(img, sigma=sigma))

    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = ensure_color_channels(gray)
    aug_images.append(gray)

    return aug_images


def augment_multiple(img):
    """Apply pairs of augmentations."""
    aug_images = []

    # Rotation + Grayscale
    M = cv2.getRotationMatrix2D((IMG_SIZE[0]//2, IMG_SIZE[1]//2), 20, 1)
    rot = cv2.warpAffine(img, M, IMG_SIZE)
    gray = ensure_color_channels(cv2.cvtColor(rot, cv2.COLOR_BGR2GRAY))
    aug_images.append(gray)

    # Contrast + Translation
    shifted = cv2.warpAffine(img, np.float32([[1, 0, 15], [0, 1, 10]]), IMG_SIZE)
    aug_images.append(adjust_brightness_contrast(shifted, contrast=1.2))

    # Scaling + Shear
    scaled = cv2.resize(img, None, fx=1.1, fy=1.1)
    M = np.array([[1, np.tan(np.radians(10)), 0], [0, 1, 0]], dtype=float)
    aug_images.append(cv2.warpAffine(scaled, M, IMG_SIZE))

    # Brightness + Contrast
    aug_images.append(adjust_brightness_contrast(img, brightness=1.1, contrast=1.2))

    # Noise + Flip
    flipped = cv2.flip(img, 1)
    aug_images.append(add_gaussian_noise(flipped, sigma=0.03))

    return aug_images

In [ ]:

# --- MAIN LOOP ---

for class_name in os.listdir(TRAIN_DATA_DIR):
    class_path = os.path.join(TRAIN_DATA_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"\nProcessing class: {class_name}")
    single_out = os.path.join(SINGLE_DIR, class_name)
    multi_out = os.path.join(MULTI_DIR, class_name)
    os.makedirs(single_out, exist_ok=True)
    os.makedirs(multi_out, exist_ok=True)

    img_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img_name in tqdm(img_files, desc=f"Augmenting {class_name}", unit="img"):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, IMG_SIZE)

        # --- Single augmentations ---
        single_augs = augment_single(img)
        base_name = os.path.splitext(img_name)[0]
        cv2.imwrite(os.path.join(single_out, f"{base_name}_orig.jpg"), img)
        for i, aug in enumerate(single_augs):
            cv2.imwrite(os.path.join(single_out, f"{base_name}_single_{i}.jpg"), aug, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

        # --- Multiple augmentations ---
        multi_augs = augment_multiple(img)
        cv2.imwrite(os.path.join(multi_out, f"{base_name}_orig.jpg"), img)
        for i, aug in enumerate(multi_augs):
            cv2.imwrite(os.path.join(multi_out, f"{base_name}_multi_{i}.jpg"), aug, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

print("\n✅ Augmentation complete! All images saved under:")
print(f"  - {SINGLE_DIR}")
print(f"  - {MULTI_DIR}")


Processing class: apple


Augmenting apple: 100%|██████████| 230/230 [00:14<00:00, 16.39img/s]



Processing class: avocado


Augmenting avocado: 100%|██████████| 230/230 [00:13<00:00, 16.88img/s]



Processing class: banana


...


Processing class: strawberries


Augmenting strawberries: 100%|██████████| 230/230 [00:14<00:00, 16.06img/s]



Processing class: watermelon


Augmenting watermelon: 100%|██████████| 230/230 [00:13<00:00, 16.44img/s]


✅ Augmentation complete! All images saved under:
  - data_augmented\single_various_augmentation
  - data_augmented\multiple_various_augmentation


# Train Model

## One transformation per time

In [7]:
# --- CONFIG ---
BATCH_SIZE = 500
EPOCHS = 100
NUM_CLASSES = 10
DATA_AUG_DIR = OUTPUT_BASE

In [ ]:


# --- Load training dataset ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    SINGLE_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# --- Resize validation dataset to match training ---
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# --- Define CNN model ---
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# --- Callbacks ---
checkpoint_path = os.path.join(f"best_model_{SINGLE_DIR}.keras")
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# --- Train ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb],
    verbose=2
)

# --- Store best validation accuracy ---
best_val_loss = max(history.history['val_loss'])

Found 75867 files belonging to 10 classes.
Found 1025 files belonging to 10 classes.
Epoch 1/40

Epoch 1: val_loss improved from inf to 1.68607, saving model to best_model_data_augmented\single_various_augmentation.keras
152/152 - 33s - 214ms/step - accuracy: 0.2156 - loss: 2.0198 - val_accuracy: 0.3541 - val_loss: 1.6861
Epoch 2/40

Epoch 2: val_loss improved from 1.68607 to 1.46763, saving model to best_model_data_augmented\single_various_augmentation.keras
152/152 - 31s - 203ms/step - accuracy: 0.4486 - loss: 1.5020 - val_accuracy: 0.4702 - val_loss: 1.4676
Epoch 3/40

Epoch 3: val_loss improved from 1.46763 to 1.42495, saving model to best_model_data_augmented\single_various_augmentation.keras
152/152 - 31s - 203ms/step - accuracy: 0.5219 - loss: 1.3253 - val_accuracy: 0.4859 - val_loss: 1.4249
Epoch 4/40

Epoch 4: val_loss improved from 1.42495 to 1.36226, saving model to best_model_data_augmented\single_various_augmentation.keras
152/152 - 31s - 204ms/step - accuracy: 0.5576 - lo

## Multiple transformations per time

In [8]:
# --- Load training dataset ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    MULTI_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# --- Resize validation dataset to match training ---
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# --- Define CNN model ---
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# --- Callbacks ---
checkpoint_path = os.path.join(f"best_model_{MULTI_DIR}.keras")
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# --- Train ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb],
    verbose=2
)

# --- Store best validation accuracy ---
best_val_loss = max(history.history['val_loss'])

Found 13794 files belonging to 10 classes.
Found 1025 files belonging to 10 classes.
Epoch 1/100

Epoch 1: val_loss improved from inf to 2.29374, saving model to best_model_data_augmented\multiple_various_augmentation.keras
28/28 - 7s - 256ms/step - accuracy: 0.0956 - loss: 2.3048 - val_accuracy: 0.1132 - val_loss: 2.2937
Epoch 2/100

Epoch 2: val_loss improved from 2.29374 to 2.20284, saving model to best_model_data_augmented\multiple_various_augmentation.keras
28/28 - 6s - 208ms/step - accuracy: 0.1185 - loss: 2.2749 - val_accuracy: 0.2010 - val_loss: 2.2028
Epoch 3/100

Epoch 3: val_loss improved from 2.20284 to 1.85336, saving model to best_model_data_augmented\multiple_various_augmentation.keras
28/28 - 6s - 206ms/step - accuracy: 0.2401 - loss: 2.0552 - val_accuracy: 0.3385 - val_loss: 1.8534
Epoch 4/100

Epoch 4: val_loss improved from 1.85336 to 1.69376, saving model to best_model_data_augmented\multiple_various_augmentation.keras
28/28 - 6s - 206ms/step - accuracy: 0.3338 - lo

In [9]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 148, 148, 5)    │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 72, 72, 5)      │           230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 5)      │           230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 15, 15, 5)      │           230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 245)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,460 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,872 (38.57 KB)

 Trainable params: 3,290 (12.85 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,582 (25.71 KB)